In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pm4py==2.1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

In [ ]:
#Changing the directory
os.chdir('/content/drive/My Drive/Process mining course/Part 2 Project-2')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import sklearn

from pydream.predictive.nap.NAP import NAP, multiclass_roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support,precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from pydream.EnhancedPN import EnhancedPN
from pm4py.objects.petri.importer import importer as pnml_importer
from backend.stat.ci_auc import calculate_auc_ci

In [ ]:
train_json = "tss_heart_log_train0.json"
valid_json ="tss_heart_log_val0.json"
test_json = "tss_heart_log_test0.json"

In [ ]:
train_algo = NAP(tss_train_file=train_json, tss_test_file=valid_json, options={"n_epochs" : 2})
train_algo.train(checkpoint_path="chk_points", name="HF_model", save_results=True)

Xtrain shape: (8403, 1920)
X2train shape: (8403, 3)
Severity train shape: (8403, 58)
X2test shape: (1531, 3)
Xtest shape: (1531, 1920)
Severity test shape: (1531, 58)
Ytrain shape: (8403, 2)
Ytest shape: (1531, 2)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Class weights :  <module 'sklearn.utils.class_weight' from '/usr/local/lib/python3.8/dist-packages/sklearn/utils/class_weight.py'>
Train on 8403 samples, validate on 1531 samples
Epoch 1/2
8400/8403 [============================>.] - ETA: 0s - loss: 0.8473 - acc: 0.1627

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


test_prec_mean 0.08001306335728282
test_rec_mean 0.5
test_fscore_mean 0.13795045045045046
test_auc_mean 0.5

Epoch 1: test_rec_mean improved from -inf to 0.50000, saving model to chk_points/HF_model_split_weights.hdf5
8403/8403 [==============================] - 17s 2ms/sample - loss: 0.8472 - acc: 0.1627 - val_loss: 1.4583 - val_acc: 0.1600 - test_acc: 0.1600 - test_prec_weighted: 0.0256 - test_rec_weighted: 0.1600 - test_loss: 1.4583 - test_fscore_weighted: 0.0442 - test_auc_weighted: 0.5000 - test_prec_mean: 0.0800 - test_rec_mean: 0.5000 - test_fscore_mean: 0.1380 - test_auc_mean: 0.5000 - lr: 0.0010
Epoch 2/2
8400/8403 [============================>.] - ETA: 0s - loss: 0.8090 - acc: 0.1594test_prec_mean 0.08001306335728282
test_rec_mean 0.5
test_fscore_mean 0.13795045045045046
test_auc_mean 0.5

Epoch 2: test_rec_mean did not improve from 0.50000
8403/8403 [==============================] - 15s 2ms/sample - loss: 0.8089 - acc: 0.1593 - val_loss: 1.3043 - val_acc: 0.1600 - test_acc

In [ ]:
algo = NAP()
algo.loadModel(path="chk_points", name="HF_model")

X_test, X2_test, severity_test, Y_test = algo.loadData(test_json)

In [ ]:
stdScaler = MinMaxScaler()
stdScaler.fit(X_test)
X_test = stdScaler.transform(X_test)
stdScaler.fit(X2_test)
X2_test = stdScaler.transform(X2_test)
stdScaler.fit(severity_test)
severity_test = stdScaler.transform(severity_test)

Y_test_int = np.array([int(i) for i in Y_test])
y_prob = algo.model.predict([X_test,X2_test,severity_test])
y_pred = np.argmax(y_prob, axis=1)

test_acc = accuracy_score(Y_test_int, y_pred, normalize=True)

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
y_true =Y_test_int
y_probscore = []
list_of_lists = []
for x in y_prob:
    list_of_lists.append(list(x))
    y_probscore.append(list(x)[1])


y_score = np.array(y_probscore)
y_pred = np.around(y_pred)
y_true = np.array(y_true)
auc, auc_cov, ci = calculate_auc_ci(y_true,y_score,y_pred, alpha=0.95)
print("AUC Score: ",auc)
print("AUC_cov: ", auc_cov)
print("confidence interval: ",ci)

*** Confusion Matrix ***
true\pred     0     1 
        0   0.0 237.0 
        1   0.0  57.0 

AUC Score:  0.7223332593086091
AUC_cov:  0.001192961125642452
confidence interval:  [0.65463754 0.79002898]
